In [1]:
from keras import models, layers
import numpy as np
import glob2
import matplotlib.pyplot as plt
import json
import pprint
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import preprocessing
import progressbar

/Users/tristanbehrens/Development/python-venvs/venv-3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset_path = "../data"

# Getting the paths for images.
glob_search_path = os.path.join(dataset_path, "**/*.jpg")
jpg_paths = glob2.glob(glob_search_path)

# Getting the paths for point clouds.
glob_search_path = os.path.join(dataset_path, "**/*.pcd")
pcd_paths = glob2.glob(glob_search_path)

# Getting the paths for personal and measurement.
glob_search_path = os.path.join(dataset_path, "**/*.json")
json_paths = glob2.glob(glob_search_path)
json_paths_personal = [json_path for json_path in json_paths if "measures" not in json_path]
json_paths_measures = [json_path for json_path in json_paths if "measures" in json_path]

In [3]:
target_names = ["height", "weight"]
def extract_targets(json_data_measure):
    targets = []
    for target_name in target_names:
        value = json_data_measure[target_name]["value"]
        targets.append(value)
    return targets


def extract_qrcode(json_data_measure):
    person_id = json_data_measure["personId"]["value"]
    json_path_personal = [json_path for json_path in json_paths_personal if person_id in json_path]
    assert len(json_path_personal) == 1
    json_path_personal = json_path_personal[0]
    json_data_personal = json.load(open(json_path_personal))
    #pprint.pprint(json_data_personal)
    qrcode = json_data_personal["qrcode"]["value"]
    return qrcode


def load_image(image_path):
    img = preprocessing.image.load_img(image_path, target_size=(160, 90))
    #img = preprocessing.image.load_img(image_path)
    img = img.rotate(-90, expand=True)
    img = np.array(img)
    return img

# A loop to extract targets.
x_images = []
y_targets = []
target_names = ["height", "weight"]
count = 0
print("Processing...")
bar = progressbar.ProgressBar(max_value=len(json_paths_measures))
for index, json_path_measure in enumerate(json_paths_measures):
    json_data_measure = json.load(open(json_path_measure))

    # Ensure manual data.
    if json_data_measure["type"]["value"] != "manual":
        continue
    
    # Extract the targets.
    targets = extract_targets(json_data_measure)

    # Get the personal file and extract the QR-code.
    qrcode = extract_qrcode(json_data_measure)
    print(qrcode)
    
    # Get all image-files.
    jpg_paths_for_qrcode = [jpg_path for jpg_path in jpg_paths if qrcode in jpg_path]
    print("Found", len(jpg_paths_for_qrcode), "JPGs.")
    print("Found", len(set(jpg_paths_for_qrcode)), "JPGs (set).")
    
    # Get all point-cloud-files.
    pcd_paths_for_qrcode = [pcd_path for pcd_path in pcd_paths if qrcode in pcd_path]
    print("Found", len(pcd_paths_for_qrcode), "PCDs.")
    
    for jpg_path in jpg_paths_for_qrcode:
        image = load_image(jpg_path)
        x_images.append(image)
        y_targets.append(targets)

        #plt.imshow(image)
        #plt.show()
        #plt.close
        #break
    
    #count += 1
    #if count == 3:
    #    break
    bar.update(index)
bar.finish()
 
x_images = np.array(x_images)
y_targets = np.array(y_targets)
print(x_images.shape)
print(y_targets.shape)

Processing...
prod_test_1
Found 58 JPGs.
Found 58 JPGs (set).
Found 0 PCDs.


NameError: name 'img_path' is not defined

# Just visualize an image.

In [ ]:



datagen = ImageDataGenerator(
      #rotation_range=40,
      #width_shift_range=0.2,
      #height_shift_range=0.2,
      #shear_range=0.2,
      #zoom_range=0.2,
      #horizontal_flip=True,
      #fill_mode='nearest'
)

# This is module with image preprocessing utilities


# We pick one image to "augment"
img_path = jpg_paths[666]

# Read the image and resize it
img = image.load_img(img_path, target_size=(150, 150))

# Convert it to a Numpy array with shape (150, 150, 3)
x = image.img_to_array(img)

# Reshape it to (1, 150, 150, 3)
x = x.reshape((1,) + x.shape)

# The .flow() command below generates batches of randomly transformed images.
# It will loop indefinitely, so we need to `break` the loop at some point!
i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    break

plt.show()